##### Pacotes Necessários 

In [1]:
!pip3 install xgboost
!pip3 install psycopg2
!pip3 install pandas
!pip3 install configparser
!pip3 install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\users\matheus orsi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\users\matheus orsi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\users\matheus orsi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\users\matheus orsi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\users\matheus orsi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import psycopg2
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from configparser import ConfigParser

c:\Users\Matheus Orsi\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


##### Conexão com a Base e Extração de Dados

In [3]:
# Função para configuração de conexão com parâmetros de acesso presentes no arquivo database.ini
# Transforma o arquivo em uma variável de objeto
def config(filename='database.ini', section='postgresql'):
    
    parser = ConfigParser()
    parser.read(filename)

    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
        print('Configurações carregadas.')
    else:
        raise Exception('Seção {0} não encontrada no arquivo {1}'.format(section, filename))
    return db

In [4]:
def connect():
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()

        retorno = []

        cursor.execute('SELECT * FROM forms')
        forms = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'forms'")
        colunas = cursor.fetchall()
        retorno.append([forms, colunas])

        cursor.execute('SELECT * FROM orders')
        orders = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'orders'")
        colunas = cursor.fetchall()
        retorno.append([orders, colunas])

        cursor.execute('SELECT * FROM order_items')
        items = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'order_items'")
        colunas = cursor.fetchall()
        retorno.append([items, colunas])

        cursor.close()
        return retorno
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Conexão encerrada.')


if __name__ == '__main__':
    retorno = connect()

Configurações carregadas.
Conexão encerrada.


In [5]:
# Atribuindo bases de dados a variáveis
forms = pd.DataFrame(retorno[0][0])
forms.columns = [item for sublist in retorno[0][1] for item in sublist]

orders = pd.DataFrame(retorno[1][0])
orders.columns = [item for sublist in retorno[1][1] for item in sublist]

order_items = pd.DataFrame(retorno[2][0])
order_items.columns = [item for sublist in retorno[2][1] for item in sublist]

##### Manipulação de dados

In [6]:
# Remoção de colunas
forms = forms.drop(['id_cliente','data','sistema_operacional','navegador','localizacao','locale','cidade','utm_source',
'caracteristica','tempo_procedimento'], axis = 1)

orders = orders[['id_pedido','target']]

order_items = order_items[['id_pedido','id_form']]

In [7]:
# Selecionando apenas os valores únicos da coluna 'id_form' da tabela 'order_items'
items_form = order_items.drop_duplicates(subset = ['id_form'])
# Left join da tabela 'forms' com a 'items_form' seguido de left join da tabela resultante com a tabela 'orders'. 
# df é a base de dados resultante
df = forms.merge(items_form, 
                 on = 'id_form', 
                 how = 'left').merge(orders, 
                                     on = 'id_pedido', 
                                     how = 'left').drop(['id_form','id_pedido'], axis = 1)

In [8]:
# Lidando com NAs
df.target.fillna(0, inplace = True)
df.dropna(axis = 0, inplace = True)
df.drop(df[df.efeitos_desejados == 'N/C'].index, axis = 0, inplace = True)
df.drop(df[df.procedimentos == 'N/C'].index, axis = 0, inplace = True)


In [9]:
# Transformação de variáveis
[df.estado, codes_estado] = pd.factorize(df.estado, sort = True)
[df.tipo_cabelo, codes_tipo_cabelo] = pd.factorize(df.tipo_cabelo, sort = True)
[df.comprimento, codes_comprimento] = pd.factorize(df.comprimento, sort = True)
[df.tipo_fios, codes_tipo_fios] = pd.factorize(df.tipo_fios, sort = True)
[df.dieta, codes_dieta] = pd.factorize(df.dieta, sort = True)
[df.atividade_fisica, codes_atividade_fisica] = pd.factorize(df.atividade_fisica, sort = True)
[df.frequencia_estresse, codes_frequencia_estresse] = pd.factorize(df.frequencia_estresse, sort = True)
[df.faixa_etaria, codes_faixa_etaria] = pd.factorize(df.faixa_etaria, sort = True)
[df.fragancia, codes_fragancia] = pd.factorize(df.fragancia, sort = True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426769 entries, 0 to 427949
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   estado               426769 non-null  int64  
 1   qtde_lavagem_semana  426769 non-null  int64  
 2   tipo_cabelo          426769 non-null  int64  
 3   comprimento          426769 non-null  int64  
 4   tipo_fios            426769 non-null  int64  
 5   procedimentos        426769 non-null  object 
 6   dieta                426769 non-null  int64  
 7   atividade_fisica     426769 non-null  int64  
 8   frequencia_estresse  426769 non-null  int64  
 9   faixa_etaria         426769 non-null  int64  
 10  efeitos_desejados    426769 non-null  object 
 11  fragancia            426769 non-null  int64  
 12  target               426769 non-null  float64
dtypes: float64(1), int64(10), object(2)
memory usage: 45.6+ MB


In [11]:
df.head()

,estado,qtde_lavagem_semana,tipo_cabelo,comprimento,tipo_fios,procedimentos,dieta,atividade_fisica,frequencia_estresse,faixa_etaria,efeitos_desejados,fragancia,target
0,264,4,1,3,1,Nenhum,0,0,0,3,Efeito 7,2,1.0
1,264,7,3,2,0,Todos acima,3,2,0,6,Efeito 1,0,0.0
2,264,2,0,0,2,Todos acima,3,3,2,2,Efeito 7,2,0.0
3,111,5,0,0,2,Todos acima,0,3,2,3,Efeito 7,2,0.0
4,260,4,2,2,2,Tintura,0,4,0,1,Efeito 7,0,0.0


In [12]:
df['efeito1'] = 0
df['efeito2'] = 0
df['efeito3'] = 0
df['efeito4'] = 0
df['efeito5'] = 0
df['efeito6'] = 0
df['efeito7'] = 0
df['efeito8'] = 0
df['efeito10'] = 0

df.loc[df['efeitos_desejados'].str.contains('Efeito 1'), 'efeito1'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 2'), 'efeito2'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 3'), 'efeito3'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 4'), 'efeito4'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 5'), 'efeito5'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 6'), 'efeito6'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 7'), 'efeito7'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 8'), 'efeito8'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 10'), 'efeito10'] = 1

In [13]:
df['nenhum'] = 0
df['tintura'] = 0
df['corte'] = 0
df['exposicao'] = 0
df['todos'] = 0

df.loc[df['procedimentos'].str.contains('Nenhum'), 'nenhum'] = 1
df.loc[df['procedimentos'].str.contains('Tintura'), 'tintura'] = 1
df.loc[df['procedimentos'].str.contains('Corte agressivo'), 'corte'] = 1
df.loc[df['procedimentos'].str.contains('Exposição ao sol'), 'exposicao'] = 1
df.loc[df['procedimentos'].str.contains('Todos acima'), 'todos'] = 1

In [14]:
df.drop(['efeitos_desejados', 'procedimentos'], axis = 1, inplace=True)

In [15]:
# Divisão entre features e variável alvo
y = df.pop('target')
X = df.copy()

# Divisão do conjunto em dados de treino e de teste (70%/30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)

In [16]:
X_train.shape

(298738, 24)

In [17]:
y_train.shape

(298738,)

In [18]:
# Treinamento e predição do modelo
model = xgb.XGBClassifier(random_state = 9)
model.fit(X_train,y_train)
preds = model.predict(X_test)

c:\Users\Matheus Orsi\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Matheus Orsi\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[21:51:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [19]:
accuracy_score(y_test,preds)

0.8737649475517648